Rede Neural

In [12]:
import pandas as pd
import numpy as np
#from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, root_mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split
#import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv("DistanciaEMediasPevs.csv", delimiter=",")

X = data.drop("MediaRendimento", axis=1)
X = X.drop("PEV", axis=1)
y = data.MediaRendimento

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
hidden_layers = (200,100,50,25)
mlp_model = MLPRegressor(
    hidden_layer_sizes=hidden_layers,
    activation='relu',
    solver='adam',
    alpha=0.0001,
    learning_rate_init=0.0003,
    max_iter=1000,
    random_state=42,
    early_stopping=True,
    n_iter_no_change=25
)

mlp_model.fit(X_train,y_train)

y_Predito = mlp_model.predict(X_test)

In [10]:
def metricas(y_test, y_Predito):
    mae = mean_absolute_error(y_test, y_Predito)
    mqe = mean_squared_error(y_test, y_Predito)
    rmqe = root_mean_squared_error(y_test, y_Predito)
    rmqle = root_mean_squared_log_error(y_test, y_Predito)
    r2 = r2_score(y_test, y_Predito)

    print(f"Mean Absolute Error: {mae}")
    print(f"Mean Squared Error: {mqe}")
    print(f"Root Mean Squared Error: {rmqe}")
    print(f"Root Mean Squared Log Error: {rmqle}")
    print(f"R Squared: {r2}")

In [ ]:
plt.scatter(y_test, y_Predito)
plt.show()

Meu modelo

Calcula a distância para todos os outros registros do conjunto

In [ ]:
from myModel import myModel

In [19]:
df = pd.read_csv("localERendimento.csv", delimiter=",", index_col="PEV")

X = df.drop("MediaRendimento", axis=1)
y = df.MediaRendimento

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

#X_train = geraTabelaComDistancias(X_train)



X_train = X_train.reset_index(drop=True)
y_train

PEV
100.0    319.095000
109.0    505.000000
2.0      140.451667
143.0    520.798500
90.0     376.921765
            ...    
31.0     365.000000
97.0     387.690000
77.0     297.658947
40.0     255.807143
134.0    452.522941
Name: MediaRendimento, Length: 122, dtype: float64

Cria a tabela com as distancias 

In [4]:
import criadorTabelaDistancia

In [17]:
def tabelaDistancia(tabela):
    ctd = criadorTabelaDistancia.criadorTabelaDistancia(X)

    distcols = ["distanciaPev" + str(i) for i in X.index]

    newDF = pd.DataFrame(columns=["latitude","longitude"] +distcols)

    for i,row in tabela.iterrows():
        distancias = ctd.calculaDistancias(row["latitude"],row["longitude"])
        cols = [row["latitude"], row["longitude"]] + distancias.tolist()
        newDF.loc[len(newDF)] = cols
        
    return newDF
    

In [20]:
X_train = tabelaDistancia(X_train)
X_test = tabelaDistancia(X_test)

Minimiza função f(x) do meu modelo

In [7]:
from scipy.optimize import minimize

In [ ]:
#print(newDF)
X_train

In [ ]:
y_train

In [40]:
model = myModel()

x0 = [2,3,4,5]
bounds = [(-100, 100)] * 4 

res = minimize(lambda params: model.calculaErro(params, X_train, y_train), x0, method="SLSQP")


print(res)
print("Melhores parâmetros:", res.x)
print("Erro mínimo:", res.fun)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 4717416.8385913605
       x: [ 3.691e+04 -4.193e+04  3.152e+04 -4.869e+04]
     nit: 50
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
    nfev: 338
    njev: 50
Melhores parâmetros: [ 36914.61896284 -41928.00135085  31523.53842563 -48694.16295165]
Erro mínimo: 4717416.8385913605


In [ ]:
y_pred = model.preve(res.x,X_test,y_train)

In [42]:
#X_test
#y_test

metricas(y_test,y_pred)

Mean Absolute Error: 191.02658484485235
Mean Squared Error: 62705.77693740272
Root Mean Squared Error: 250.41121567813755
Root Mean Squared Log Error: 0.6131698392990476
R Squared: -0.7953495082565176
